Importing the required libraries 

In [32]:
import numpy as np #Linear Algebra
import pandas as pd #Read csv file
import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split #Split the data for scoring purposes
from sklearn.linear_model import Lasso #Lasso algorithm
from sklearn.metrics import r2_score, mean_absolute_error #Scoring systems

Reading csv file and converting every string value in the file to numbers so that Lasso algorithm understand it.

df = pd.read_csv('LBNL_file.csv', low_memory=False) #Read the csv file


df.head(10)

df.describe(percentiles=(0.1,0.2,0.25,0.5,0.75,0.90),include=None)


for column in df.columns:
    #Within each column, if there is a string, convert the strings to numbers and map them out to the dataframe
    if isinstance(df.iloc[[0]][column][0], str):
        unique_values = np.unique(df[column])
        d = {unique: index+1 for index, unique in enumerate(unique_values)}
        df[column] = df[column].map(d)

def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) 
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

plotPerColumnDistribution(df, 15, 5)

df.shape

X and y and split them into train(70% of data) and test(30% of data) 

#Our y will be the installed price, and our X will be everything but the price
X = df.drop('total_installed_price', axis=1)
y = df['total_installed_price']

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Now, we will apply the Lasso algorithm with a 0.1 alpha to our train data, and score our algorithm based on how it does in the test data.

#Create our model and get the predictions based on X_test
model = Lasso(alpha=0.1).fit(X_train, y_train)
predictions = model.predict(X_test)

print(mean_absolute_error(y_test, predictions)) #My mean absolute error: 26870.59197623797 
print(r2_score(y_test, predictions)) #My r2_score: 0.21133233070095414

There is a room for improvement in the algorithm,I will try to use others algorithms also soon.